https://stackoverflow.com/questions/12761510/python-how-can-i-calculate-the-average-word-length-in-a-sentence-using-the-spl

https://pythonprogramming.net/combine-classifier-algorithms-nltk-tutorial/?completed=/sklearn-scikit-learn-nltk-tutorial/

http://streamhacker.com/2012/11/22/text-classification-sentiment-analysis-nltk-scikitlearn/

http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sample-pipeline-for-text-feature-extraction-and-evaluation

In [2]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
import logging

ran.seed(5125)

from pprint import *
from time import *

%load_ext cython
%load_ext cythonmagic
%matplotlib inline
%matplotlib notebook

import rpy2.robjects as robjects

from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.grid_search import *

/usr/local/lib/python3.4/dist-packages/IPython/extensions/cythonmagic.py:21: UserWarning: The Cython magic has been moved to the Cython package
  warnings.warn("""The Cython magic has been moved to the Cython package""")
/usr/local/lib/python3.4/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [7]:
data = pd.read_csv("../raw_data/data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head(n=1)

data_utf8 = pd.read_csv("../raw_data/data_utf8.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, engine = "python")
data_utf8.head()

,"﻿""#AUTHID""",STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [3]:
word = "likes the sound of thunder."
print(sum(c != ' ' for c in word))
data["Lenght_of_Rows_withSpaces"] = np.nan
data["Lenght_of_Rows_withoutSpaces"] = np.nan
data["Word_Lenght"] = np.nan

data.head(n=1)

23


,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,...,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY,Lenght_of_Rows_withSpaces,Lenght_of_Rows_withoutSpaces,Word_Lenght
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,...,180,14861.6,93.29,0.03,15661,0.49,0.1,NaN,NaN,NaN


In [4]:
d = {'n': "no", 'y': "yes"}

# http://stackoverflow.com/a/17702781
data = data.replace(d)

neu = data[["#AUTHID","STATUS","cNEU"]]
ext = data[["#AUTHID","STATUS","cEXT"]]
agr = data[["#AUTHID","STATUS","cAGR"]]
con = data[["#AUTHID","STATUS","cCON"]]
opn = data[["#AUTHID","STATUS","cOPN"]]

In [9]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66)
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66)
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66)
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66)
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66)

In [20]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])


In [21]:
# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

In [25]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


clas_fit_SVC = pipeline.fit(train_neu["STATUS"], train_neu["cNEU"]).predict(test_neu["cNEU"])
scores_SVC = cross_validation.cross_val_score(pipeline, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(neu["STATUS"], neu["cNEU"])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

[Parallel(n_jobs=-1)]: Done  49 tasks       | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 199 tasks       | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 449 tasks       | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 799 tasks       | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 1249 tasks       | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 1799 tasks       | elapsed: 26.8min
[Parallel(n_jobs=-1)]: Done 2449 tasks       | elapsed: 34.9min
[Parallel(n_jobs=-1)]: Done 3199 tasks       | elapsed: 44.8min


Accuracy rbf kernel SVC: 0.582 (+/- 0.01)
Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (1e-05, 1e-06),
 'clf__n_iter': (10, 50, 80),
 'clf__penalty': ('l2', 'elasticnet'),
 'tfidf__norm': ('l1', 'l2'),
 'tfidf__use_idf': (True, False),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__max_features': (None, 5000, 10000, 50000),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 3 folds for each of 1152 candidates, totalling 3456 fits
done in 2911.556s

Best score: 0.607
Best parameters set:
	clf__alpha: 1e-05
	clf__n_iter: 10
	clf__penalty: 'l2'
	tfidf__norm: 'l1'
	tfidf__use_idf: False
	vect__max_df: 1.0
	vect__max_features: None
	vect__ngram_range: (1, 2)


[Parallel(n_jobs=-1)]: Done 3456 out of 3456 | elapsed: 48.5min finished


### Lexical and other features

In [13]:
def SyntacticFeatures(chapters):
    """
    Extract feature vector for part of speech frequencies
    """
    list_of_rows_CD = [l.split("\n")[0] for l in chapters]
    
    def token_to_pos(ch):
        tokens = n.word_tokenize(ch)
        #print(tokens)
        return [p[1] for p in n.pos_tag(tokens)]

    chapters_pos = [token_to_pos(kCD) for kCD in list_of_rows_CD]
    print(chapters_pos)
    
    pos_list = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS']
    fvs_syntax = np.array([[ch.count(pos) for pos in pos_list]
                           for ch in chapters_pos]).astype(np.float64)
    
    print(fvs_syntax)
    # normalise by dividing each row by number of tokens in the chapter
    fvs_syntax /= np.c_[np.array([len(ch) for ch in chapters_pos])]

    return fvs_syntax

feature_sets = None
feature_sets = list(SyntacticFeatures(neu["STATUS"]))
feature_sets

KeyboardInterrupt: 

In [ ]:
def LexicalFeatures(chapters):

    sentence_tokenizer = n.data.load('tokenizers/punkt/english.pickle')
    word_tokenizer = n.tokenize.RegexpTokenizer(r'\w+')
    
    for row in chapters.iterrows():
        # print(row)
        
        num_chapters = len(row)
        tokens = n.word_tokenize(row.lower())
        words = word_tokenizer.tokenize(row.lower())
        sentences = sentence_tokenizer.tokenize(row)
        vocab = set(words)
        words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                           for s in sentences])
        print("druhy pokus:", num_chapters, tokens, words, sentences, vocab, words_per_sentence)

        # create an array of zeros https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.zeros.html
        #fvs_lexical = np.zeros((len(chapters), 3), np.float64)
        #fvs_punct = np.zeros((len(chapters), 3), np.float64)

        #print("testing:", num_chapters, chapters, fvs_lexical, fvs_punct)

#         for e, ch_text in enumerate(num_chapters):
            # note: the nltk.word_tokenize includes punctuation
            

#             # average number of words per sentence
#             fvs_lexical[e, 0] = words_per_sentence.mean()
#             # sentence length variation
#             fvs_lexical[e, 1] = words_per_sentence.std()
#             # Lexical diversity
#             fvs_lexical[e, 2] = len(vocab) / float(len(words))

#             # Commas per sentence
#             fvs_punct[e, 0] = tokens.count(',') / float(len(sentences))
#             # Semicolons per sentence
#             fvs_punct[e, 1] = tokens.count(';') / float(len(sentences))
#             # Colons per sentence
#             fvs_punct[e, 2] = tokens.count(':') / float(len(sentences))


        # apply whitening to decorrelate the features
#         fvs_lexical = whiten(fvs_lexical)
#         fvs_punct = whiten(fvs_punct)

#     return fvs_lexical, fvs_punct

In [ ]:
#LexicalFeatures(neu[["STATUS"]])
word_tokenizer = n.tokenize.RegexpTokenizer(r'\w+')
words_per_sentence_AVG = word_tokenizer.tokenize(neu["STATUS"].str.lower())
feature_sets = None
feature_sets = list(LexicalFeatures(neu[["STATUS"]]))
feature_sets

### Sentence lenght 

In [ ]:
# df1['e'] = Series(np.random.randn(sLength), index=df1.index) .loc[:,'f']
for index, sentence in data.iterrows():
    filtered = ''.join(filter(lambda x: x not in '".,;!?-', sentence["STATUS"]))
    words = [word for word in filtered.split() if word]
    try:
        avg = sum(map(len, words))/len(words)
    except ZeroDivisionError:
        avg = 0
    data["Word_Lenght"][index] = "%0.1f" % avg

### Average word length in a sentence

In [ ]:
i=0
for sentence in zip(data["STATUS"]):
    #var1 = data[["STATUS"]][sentence.index]
    data["Lenght_of_Rows_withoutSpaces"][i] = sum(c != ' ' for c in sentence[0])
    i = i+1
    #data["Lenght_of_Rows_withSpaces"][index] = len(sentence["STATUS"])
    #data["Lenght_of_Rows_withSpaces"].apply(sum(c != ' ' for c in sentence["STATUS"]), axis=1)
data.head(n=2)